In [1]:
#Chat Bot Training
# importing the required modules.
import random
import json
import pickle
import numpy as np
import nltk
  
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [2]:
lemmatizer = WordNetLemmatizer()
  
# reading the json.intents file
intents = json.loads(open("intents.json").read())
  
# creating empty lists to store data
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # separating words from patterns
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # and adding them to words list
          
        # associating patterns with respective tags
        documents.append(((word_list), intent['tag']))

        # adding tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
#storing the root forms or lemma
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(list(set(words)))

# save words and classes to files
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [3]:
# creating training data
training = []
output_empty = [0] * len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # making a copy of output_empty list for each pattern
    output_row = list (output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\anest\AppData\Local\Temp/ipykernel_11948/1915310979.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [4]:
# creating a sequential model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# compiling the model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

c:\Users\anest\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 12s 23ms/step - loss: 1.9831 - accuracy: 0.1000
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 1.9668 - accuracy: 0.0667
Epoch 3/200
6/6 [==============================] - 0s 9ms/step - loss: 1.7776 - accuracy: 0.2333
Epoch 4/200
6/6 [==============================] - 0s 11ms/step - loss: 1.7082 - accuracy: 0.3667
Epoch 5/200
6/6 [==============================] - 0s 11ms/step - loss: 1.6894 - accuracy: 0.2333
Epoch 6/200
6/6 [==============================] - 0s 16ms/step - loss: 1.5255 - accuracy: 0.4333
Epoch 7/200
6/6 [==============================] - 0s 17ms/step - loss: 1.4624 - accuracy: 0.3333
Epoch 8/200
6/6 [==============================] - 0s 13ms/step - loss: 1.4599 - accuracy: 0.5000
Epoch 9/200
6/6 [==============================] - 0s 12ms/step - loss: 1.2910 - accuracy: 0.4667
Epoch 10/200
6/6 [==============================] - 0s 14ms/step - loss: 1.2343 - accuracy: 0.5333
Epoch 11/200
6/6 [==